In [1]:
import numpy as np
import pandas as pd
import splat
import popsims
from astropy.coordinates import SkyCoord, Galactic
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
#sns.set_style("darkgrid", {"axes.facecolor": ".95"})

#plt.style.use('fivethirtyeight')
#plt.style.use('dark_background')

from  matplotlib.colors import Normalize
import astropy.units as u
import popsims
import matplotlib
from tqdm import tqdm
from tqdm import tqdm
import matplotlib as mpl 
#mpl.rcParams['axes.labelsize'] = 20
#mpl.rcParams['xtick.labelsize'] =20
#mpl.rcParams['ytick.labelsize'] = 20
#mpl.rcParams['legend.fontsize'] = 20
#mpl.rcParams['figure.titlesize'] = 20

from astropy.coordinates import Angle
import astropy.table as astrotable

splat.initializeStandards(all=True)
from scipy.interpolate import griddata, interpn
import splat.empirical as spe
%matplotlib inline

Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database
Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database


In [2]:
from popsims.widefields import  WideField, create_mixed_population,\
transform_tocylindrical
from astropy.coordinates import  SkyCoord
import astropy.units as u

In [3]:
coord = SkyCoord(ra=62.0*u.degree, dec=-37.0*u.degree, frame='icrs')
radius = 1* u.deg

In [4]:
grid=np.arange(15, 32)

In [5]:
mag_keys=[ 'LSST_R', 'LSST_I', 'LSST_Z']
max_distances={}
for g in grid:
    pol=popsims.POLYNOMIALS['absmags']['dwarfs']['LSST_Z']['fit']
    max_distances[g]=popsims.get_distance(pol(g), 25.)

In [6]:
narro_field=popsims.Pointing(coord=coord)
halo_field=popsims.Pointing(coord=coord, density='spheroid')

In [7]:
narro_field.mag_limits=dict(zip([k for k in mag_keys],\
         [(0,  24) for k in  mag_keys]))
thin_dist_list =narro_field.draw_distances(0.1, max_distances[15],350,  nsample=1e5)
thick_dist_list =narro_field.draw_distances(0.1, max_distances[15],900,  nsample=1e5)
halo_dist_list =halo_field.draw_distances(0.1, max_distances[15],900,  nsample=1e5)

In [8]:
def create_pop(bfraction=None,\
                     model=None,  nsample=None,
                     dmax=None, l=None, b=None, \
               absmag_keys=['WFIRST_WFIJ'], 
              population='thin_disk', dmax_dict=None):
   
    if population =='thin_disk':
        df=popsims.make_systems(model=model, bfraction=bfraction,\
                            mass_age_range=[0.01, 0.15, 0., 8.0], \
                                nsample=nsample,
                                recompute=True)
        df=df[df.spt.between(grid[0], grid[-1])]
        #df=df.sample(n=len(ds))

        #assign distances
        df['d']= np.random.choice(thin_dist_list, len(df))
        df['l']=l
        df['b']=b
        r, z=transform_tocylindrical(df.l.values, df.b.values, df.d.values)
        df['r']=r
        df['z']=z
        #us, vs, ws=get_uvw(df.age, kind=population)
    
        #df['U']=us
        #df['V']=vs
        #df['W']=ws
        dff=popsims.pop_mags_from_type(df.spt.values, d=df.d.values, \
                                       keys=absmag_keys, object_type='dwarfs').join(df)
    if population =='thick_disk':
        df=popsims.make_systems(model=model, bfraction=bfraction,\
                                 nsample=nsample,
                            mass_age_range=[0.01, 0.15, 8., 13.0], recompute=True)
        df=df[df.spt.between(grid[0], grid[-1])]
        #df=df.sample(n=len(ds))
        #assign distances
          #assign distances
        df['d']= np.random.choice(thick_dist_list, len(df))
        df['l']=l
        df['b']=b
        r, z=transform_tocylindrical(df.l.values, df.b.values, df.d.values)
        df['r']=r
        df['z']=z
        #us, vs, ws=get_uvw(df.age, kind=population)
    
        #df['U']=us
        #df['V']=vs
        #df['W']=ws
        dff=popsims.pop_mags_from_type(df.spt.values, d=df.d.values, \
                                       keys=absmag_keys, object_type='subdwarfs').join(df)
    if population =='halo':
        df=popsims.make_systems(model=model, bfraction=bfraction,\
                                 nsample=nsample,
                            mass_age_range=[0.01, 0.15, 10., 13.0], recompute=True)
        df=df[df.spt.between(grid[0], grid[-1])]
        #df=df.sample(n=len(ds))
        
        #assign distances
        df['d']= np.random.choice(halo_dist_list, len(df))
        df['l']=l
        df['b']=b
        r, z=transform_tocylindrical(df.l.values, df.b.values, df.d.values)
        df['r']=r
        df['z']=z
        #us, vs, ws=get_uvw(df.age, z= df.z, kind='halo')
        #df['U']=us
        #df['V']=vs
        #df['W']=ws
      
        dff=popsims.pop_mags_from_type(df.spt.values, d=df.d.values, \
                                       keys=absmag_keys, object_type='subdwarfs').join(df)
    return dff

In [9]:
tqdm.pandas()
df_thin=df=create_pop(bfraction=0.2,\
                         model='burrows1997',\
                             l=coord.galactic.l, b=coord.galactic.b,\
                           absmag_keys=mag_keys, \
                      nsample=1e5, dmax_dict=max_distances)

In [10]:
tqdm.pandas()
df_thick=df=create_pop(bfraction=0.2,\
                         model='burrows1997',\
                             l=coord.galactic.l, b=coord.galactic.b,
                           absmag_keys=mag_keys, \
                      nsample=1e5, dmax_dict=max_distances,
                         population='thick_disk')

In [11]:
tqdm.pandas()
df_halo=df=create_pop(bfraction=0.2,\
                         model='burrows1997',\
                              l=coord.galactic.l, b=coord.galactic.b,
                           absmag_keys=mag_keys, \
                      nsample=1e5, dmax_dict=max_distances,
                         population='halo')


In [12]:
#df_halo

In [13]:
lsst_dp0.columns

NameError: name 'lsst_dp0' is not defined

In [ ]:
#MAKE Y CUT AT 24
lsst_dp0=lsst_dp0[lsst_dp0.mag_z<25]
df_halo_f=df_halo[np.logical_and(df_halo.LSST_Z<25, df_halo.spt.between(10, 28))]
df_thin_f=df_thin[np.logical_and(df_thin.LSST_Z<25, df_thin.spt.between(17, 27))]

In [ ]:
len(lsst_dp0), len(df_halo_f), len(df_thin_f)

In [ ]:
h=plt.hist(df_halo.abs_LSST_R-df_halo.abs_LSST_I)

In [ ]:
fig, ax=plt.subplots(figsize=(8, 6))
ax.scatter((lsst_dp0.mag_r-lsst_dp0.mag_i)[lsst_dp0.truth_type==1], \
           (lsst_dp0.mag_r-lsst_dp0.mag_z)[lsst_dp0.truth_type==1], \
           s=1, c='k', alpha=.1, label='DP0 Galaxy')
ax.scatter((lsst_dp0.mag_r-lsst_dp0.mag_i)[lsst_dp0.truth_type==2], \
           (lsst_dp0.mag_r-lsst_dp0.mag_z)[lsst_dp0.truth_type==2], \
           s=1, c='r', alpha=1., label='DP0 Star')
#ax.scatter(df_thick.LSST_I-df_thick.LSST_Z, df_thick.LSST_Y- df_thick.LSST_I)
ax.scatter(df_thin_f.LSST_R-df_thin_f.LSST_I, df_thin_f.LSST_I- df_thin_f.LSST_Z, s=5, 
           alpha=0.5, label='dwarfs')
ax.scatter(df_halo_f.LSST_R-df_halo_f.LSST_I, df_halo_f.LSST_I- df_halo_f.LSST_Z, s=10, \
           alpha=0.5, c='b',  label='halo subdwarfs')
ax.set(xlim=[-5, 5], ylim=[-1, 5], xlabel='r-i', ylabel='i-z')
ax.minorticks_on()
plt.legend(loc='upper right', fontsize=14, markerscale=2)
plt.savefig(popsims.FIGURES+'/lsst_dpo_ri_iz_mag25.jpeg', rasterized=True)

In [ ]:
zhang_subds=pd.read_pickle(popsims.DATA_FOLDER+'/combined_sd_templates.pkl')
optical_stds=pd.read_pickle(popsims.DATA_FOLDER+'/combined_optical_templates.pkl')
from astropy.io import ascii
import glob

In [ ]:
np.unique(optical_stds.dropna().sptype.values)

In [ ]:

def read_filter(filtr):
    file = glob.glob('/users/caganze/research/splat/resources/filters/*{}*.txt'.format(filtr))[0]
    return ascii.read(file).to_pandas()
    

In [ ]:
np.unique(zhang_subds.sptype)

(1) select all the sources z < 20 (so they are also picked up at i band - what is i limit?);
(2) use i-z to compute spectral type and M_z;
(3) estimate distance from z & M_z; (4)

In [ ]:
from shapey import Box

In [ ]:
b=Box()
b.vertices=[(-6.0, 0.0), (-5.0, 4.0), (0.0, 4.0), (0.0, 0.0), (-6.0, 0.0)]

In [ ]:
selected=lsst_dp0[ucd_bool]

In [ ]:
fig, ax=plt.subplots(figsize=(8, 6))

ax.scatter((selected.mag_r-selected.mag_i),\
           (selected.mag_r-selected.mag_z), \
           s=10, c='k', alpha=1., label='DP0 Selected Sample')

ax.scatter(-df_thin_f.LSST_R+df_thin_f.LSST_I, df_thin_f.LSST_I- df_thin_f.LSST_Z, s=5, 
           alpha=0.5, label='dwarfs')
ax.scatter(-df_halo_f.LSST_R+df_halo_f.LSST_I, df_halo_f.LSST_I- df_halo_f.LSST_Z, s=10, \
           alpha=0.5, c='b',  label='halo subdwarfs')
ax.set(xlim=[-6, 5], ylim=[-1, 5], xlabel='r-i', ylabel='i-z')
ax.minorticks_on()
plt.legend(loc='upper right', fontsize=14, markerscale=2)
plt.savefig(popsims.FIGURES+'/lsst_dpo_ri_iz_selection.jpeg', rasterized=True)

In [ ]:
plt.plot(np.linspace(10, 5, 1000), pol(np.linspace(0, 5, 1000)))

In [ ]:
selected.